## Step 1. Download/Import an SR Product 

In [ ]:
filename = "/Users/rugi/Documents/Machine Learning RS/Planet Imagery/Neubrandenburg/Neubrandenburg_psscene_analytic_sr_udm2/files/PSScene/20220623_091358_90_242d/analytic_sr_udm2/20220623_091358_90_242d_3B_AnalyticMS_SR.tif"

## Step 2. Extract the data from the red and near-infrared bands 

In this step, you'll use [Rasterio](https://github.com/mapbox/rasterio), a Python library for reading and writing geospatial raster datasets, to open the raster image you downloaded (the .tif file). Then you'll extract the data from the red and near-infrared bands and load the band data into arrays that you can manipulate using Python's [NumPy](http://www.numpy.org/) libary. 

In [ ]:
import rasterio
import numpy as np



# Load red and NIR bands - note all PlanetScope 4-band images have band order BGRN
with rasterio.open(filename) as src:
    band_red = src.read(3)

with rasterio.open(filename) as src:
    band_nir = src.read(4)

## Step 3. Perform the NDVI calculation

Next, calculate NDVI through subtraction and division of the values stored in the NumPy arrays. This calculation gives NDVI values that range from -1 to 1. Values closer to 1 indicate a greater density of vegetation or higher level of "greenness."

In [ ]:
# Allow division by zero
np.seterr(divide='ignore', invalid='ignore')

# Calculate NDVI. This is the equation at the top of this guide expressed in code
ndvi = (band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red)

In [ ]:
# check range NDVI values, excluding NaN
np.nanmin(ndvi), np.nanmax(ndvi)

## Step 5. Save the NDVI image 

saving the calculated NDVI values to a new image file, making sure the new image file has the same geospatial metadata as the original GeoTIFF imported.

In [ ]:
# Set spatial characteristics of the output object to mirror the input
kwargs = src.meta
kwargs.update(
    dtype=rasterio.float32,
    count = 1)

# Write band calculations to a new raster file
with rasterio.open('output/nb_ndvi.tif', 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))

## Step 6. Applying a color scheme to visualize the NDVI values

visualizing the NDVI map values with matplotlib [Matplotlib](https://matplotlib.org/)  calculated from the SR Image. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors


class MidpointNormalize(colors.Normalize):
   
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        self.midpoint = midpoint
        colors.Normalize.__init__(self, vmin, vmax, clip)

    def __call__(self, value, clip=None):
        # I'm ignoring masked values and all kinds of edge cases to make a
        # simple example...
        x, y = [self.vmin, self.midpoint, self.vmax], [0, 0.5, 1]
        return np.ma.masked_array(np.interp(value, x, y), np.isnan(value))


# Set min/max values from NDVI range for image (excluding NAN)
# set midpoint according to how NDVI is interpreted: https://earthobservatory.nasa.gov/Features/MeasuringVegetation/
min=np.nanmin(ndvi)
max=np.nanmax(ndvi)
mid=0.1

fig = plt.figure(figsize=(20,10))
ax = fig.add_subplot(111)

# diverging color scheme chosen from https://matplotlib.org/users/colormaps.html
cmap = plt.cm.RdYlGn 

cax = ax.imshow(ndvi, cmap=cmap, clim=(min, max), norm=MidpointNormalize(midpoint=mid,vmin=min, vmax=max))

ax.axis('off')
ax.set_title('Normalized Difference Vegetation Index', fontsize=18, fontweight='bold')

cbar = fig.colorbar(cax, orientation='horizontal', shrink=0.65)

fig.savefig("output/nb_ndvi-fig.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

plt.show()

## 7. Generating a histogram of NDVI values

 Generating a histogram of NDVI values

In [ ]:
fig2 = plt.figure(figsize=(10,10))
ax = fig2.add_subplot(111)

plt.title("NDVI Histogram", fontsize=15, fontweight='bold')
plt.xlabel("NDVI Values", fontsize=10)
plt.ylabel("Number of Pixels", fontsize=10)


x = ndvi[~np.isnan(ndvi)]
numBins = 20
ax.hist(x,numBins,color='green',alpha=0.8)

fig2.savefig("output/nb_ndvi-histogram.png", dpi=300, bbox_inches='tight', pad_inches=0.7)

plt.show()